In [46]:
import numpy as np

In [47]:

class ExtendedKalmanFilter:
    def __init__(self, initial_state, initial_covariance, process_noise_covariance, measurement_noise_covariance,
                 state_transition_function, observation_function, state_transition_jacobian, observation_jacobian, error_jacobian):
        self.state_estimate = initial_state
        self.covariance_estimate = initial_covariance
        self.Q = process_noise_covariance
        self.R = measurement_noise_covariance
        self.state_transition_function = state_transition_function
        self.observation_function = observation_function
        self.state_transition_jacobian = state_transition_jacobian
        self.observation_jacobian = observation_jacobian
        self.error_jacobian = error_jacobian

    def predict(self):
        # Predict step
        self.state_estimate = self.state_transition_function(self.state_estimate)
        F = self.state_transition_jacobian(self.state_estimate)
        self.covariance_estimate = np.dot(np.dot(F, self.covariance_estimate), F.T) + self.Q # np.dot(np.dot(self.error_jacobian, self.Q), (self.Q).T) #added the error matrix part

    def update(self, observed_measurement):
        # Update step
        H = self.observation_jacobian(self.state_estimate)
        y = observed_measurement - self.observation_function(self.state_estimate)
        print(H.shape, (self.covariance_estimate).shape, (self.R).shape)
        S = np.dot(np.dot(H, self.covariance_estimate), H.T) + self.R
        K = np.dot(np.dot(self.covariance_estimate, H.T), np.linalg.inv(S))
        self.state_estimate = self.state_estimate + np.dot(K, y)
        I = np.eye(self.covariance_estimate.shape[0])
        self.covariance_estimate = np.dot((I - np.dot(K, H)), self.covariance_estimate)




In [48]:
# Example usage
if __name__ == "__main__":
    # Define initial values and matrices
    initial_state = np.zeros(12)
    initial_covariance = np.zeros((12, 12))
    process_noise_covariance = np.eye(12)
    measurement_noise_covariance = np.eye(3)

    # Define the state transition function and its Jacobian
    # def state_transition_function(state):
    #     # Implement the provided non-linear state transition function here
    #     pass

    def state_transition_function(state):
        x, y, z, psi, u, v, w, r, b_x, b_y, b_z, b_psi = state

        tau1, tau2, tau3, tau4 = 15, 12, 0, 0  # Updated values
        # Wiener processes for bx, by, bz, and bpsi
        dt = 0.01
        b_x += np.random.normal(scale=np.sqrt(dt))
        b_y += np.random.normal(scale=np.sqrt(dt))
        b_z += np.random.normal(scale=np.sqrt(dt))
        b_psi += np.random.normal(scale=np.sqrt(dt))
        # Zero-mean Gaussian noise for wbx, wby, wbz, and wbpsi
        wb_x = np.random.normal(scale=np.sqrt(dt))
        wb_y = np.random.normal(scale=np.sqrt(dt))
        wb_z = np.random.normal(scale=np.sqrt(dt))
        wb_psi = np.random.normal(scale=np.sqrt(dt))
        # Given state transition function
        return np.array([
            np.cos(psi) * u - np.sin(psi) * v,
            np.cos(psi) * v + np.sin(psi) * u,
            w,
            r,
            (1315892839236077 * tau1) / 18014398509481984 - (1212595251356045 * u) / 9007199254740992 - 0.78804538217602188355025252290643 * r * v + (1315892839236077 * np.cos(psi) * b_x) / 18014398509481984 - (1315892839236077 * np.sin(psi) * b_y) / 18014398509481984,
            (5149134226861935 * tau2) / 144115188075855872 - (8187638334133163 * v) / 72057594037927936 - 0.12541456625279259105718470889351 * r * u + (5149134226861935 * np.cos(psi) * b_y) / 144115188075855872 + (5149134226861935 * np.sin(psi) * b_x) / 144115188075855872,
            (7356743338895695 * b_z) / 144115188075855872 + (7356743338895695 * tau3) / 144115188075855872 + (1389688816717397 * w) / 72057594037927936,
            (1135129035283523 * b_psi) / 281474976710656 + (6305868816807027 * r) / 18014398509481984 + (1135129035283523 * tau4) / 281474976710656 + 57.662298912638639724630477211478 * u * v,
            wb_x - 10 * b_x,
            wb_y - 10 * b_y,
            wb_z - 10 * b_z,
            wb_psi - 10 * b_psi
    ])

    # def state_transition_jacobian(state):
    #     # Implement the provided state transition Jacobian matrix here
    #     pass

    # Define the state transition Jacobian matrix
    def state_transition_jacobian(state):
        psi = state[3]
        u, v, r, b_x, b_y = state[4], state[5], state[7], state[8], state[9]
        return np.array([
            [0, 0, 0, -1.0 * np.cos(psi) * v - 1.0 * np.sin(psi) * u, np.cos(psi), -1.0 * np.sin(psi), 0, 0, 0, 0, 0, 0],
            [0, 0, 0, np.cos(psi) * u - 1.0 * np.sin(psi) * v, np.sin(psi), np.cos(psi), 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0],
            [0, 0, 0, -0.07305 * np.cos(psi) * b_y - 0.07305 * np.sin(psi) * b_x, -0.1346, -0.788 * r, 0, -0.788 * v, 0.07305 * np.cos(psi), -0.07305 * np.sin(psi), 0, 0],
            [0, 0, 0, 0.03573 * np.cos(psi) * b_x - 0.03573 * np.sin(psi) * b_y, -0.1254 * r, -0.1136, 0, -0.1254 * u, 0.03573 * np.sin(psi), 0.03573 * np.cos(psi), 0, 0],
            [0, 0, 0, 0, 0, 0, 0.01929, 0, 0, 0, 0.05105, 0],
            [0, 0, 0, 0, 57.66 * v, 57.66 * u, 0, 0.35, 0, 0, 0, 4.033],
            [0, 0, 0, 0, 0, 0, 0, 0, -10.0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, -10.0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -10.0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -10.0]
        ])


    # Define the observation function and its Jacobian
    # def observation_function(state):
    #     return state[:4]

    def observation_function(state):
        z, psi, r = state[2], state[3], state[7]
        return np.array([z, psi, r])

    # def observation_jacobian(state):
    #     return np.eye(4)

    # Define the observation Jacobian matrix
    def observation_jacobian(state):
        return np.array([
            [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
        ])

    error_jacobian = np.vstack((np.zeros((8, 4)), np.eye(4)))


    # Create an instance of the ExtendedKalmanFilter class
    ekf = ExtendedKalmanFilter(initial_state, initial_covariance, process_noise_covariance, measurement_noise_covariance,
                               state_transition_function, observation_function, state_transition_jacobian, observation_jacobian, error_jacobian)

    # Perform EKF iterations (predict and update)
    num_iterations = 8
    for _ in range(num_iterations):
        ekf.predict()
        observed_measurement = np.array([10.0, 86.0, 3.6])  # Simulated measurement
        ekf.update(observed_measurement)
    # Run the predict and update steps
    # ekf.predict()
    # # Provide the observed measurement for the update step
    # observed_measurement = np.array([1, 2, 3])  # Replace with actual observed measurement
    # ekf.update(observed_measurement)

    # Print the final state estimate and covariance estimate
    print("Final State Estimate:")
    print(ekf.state_estimate)
    print("Final Covariance Estimate:")
    print(ekf.covariance_estimate)




(3, 12) (12, 12) (3, 3)
(3, 12) (12, 12) (3, 3)
(3, 12) (12, 12) (3, 3)
(3, 12) (12, 12) (3, 3)
(3, 12) (12, 12) (3, 3)
(3, 12) (12, 12) (3, 3)
(3, 12) (12, 12) (3, 3)
(3, 12) (12, 12) (3, 3)
Final State Estimate:
[nan nan nan nan nan nan nan nan nan nan nan nan]
Final Covariance Estimate:
[[nan nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan]
 [nan nan nan nan nan nan nan nan nan nan nan nan]]
